In [0]:
#Create Dataset
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# Data setup
data = [
    (101, "Laptop", "Electronics", 55000, 10),
    (102, "Smartphone", "Electronics", 30000, 25),
    (103, "Chair", "Furniture", 2500, 50),
    (104, "Book", "Stationery", 400, 200),
    (105, "Headphones", "Electronics", 1500, 100),
    (106, "Table", "Furniture", 3200, 40),
    (107, "Pen", "Stationery", 20, 500),
    (108, "Monitor", "Electronics", 12000, 15),
    (109, "Notebook", "Stationery", 60, 300),
    (110, "Sofa", "Furniture", 45000, 5)
]

columns = ["product_id", "product_name", "category", "price", "quantity"]

df = spark.createDataFrame(data, columns)
df.show()


+----------+------------+-----------+-----+--------+
|product_id|product_name|   category|price|quantity|
+----------+------------+-----------+-----+--------+
|       101|      Laptop|Electronics|55000|      10|
|       102|  Smartphone|Electronics|30000|      25|
|       103|       Chair|  Furniture| 2500|      50|
|       104|        Book| Stationery|  400|     200|
|       105|  Headphones|Electronics| 1500|     100|
|       106|       Table|  Furniture| 3200|      40|
|       107|         Pen| Stationery|   20|     500|
|       108|     Monitor|Electronics|12000|      15|
|       109|    Notebook| Stationery|   60|     300|
|       110|        Sofa|  Furniture|45000|       5|
+----------+------------+-----------+-----+--------+



In [0]:
#1: Read from CSV & Print Schema
df.write.mode("overwrite").option("header", True).csv("dbfs:/tmp/products_csv")

df_csv = spark.read.option("header", True).option("inferSchema", True).csv("dbfs:/tmp/products_csv")
df_csv.printSchema()


root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- quantity: integer (nullable = true)



In [0]:
#2: Read from JSON & Compare Schema
df.write.mode("overwrite").json("dbfs:/tmp/products_json")

df_json = spark.read.option("inferSchema", True).json("dbfs:/tmp/products_json")
df_json.printSchema()


root
 |-- category: string (nullable = true)
 |-- price: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity: long (nullable = true)



In [0]:
#3: Convert CSV to Parquet
df_csv.write.mode("overwrite").parquet("dbfs:/tmp/products_parquet")


In [0]:
#4: Measure File Sizes
def get_size(path):
    return sum(f.size for f in dbutils.fs.ls(path))

print("CSV Size:", get_size("dbfs:/tmp/products_csv"), "bytes")
print("JSON Size:", get_size("dbfs:/tmp/products_json"), "bytes")
print("Parquet Size:", get_size("dbfs:/tmp/products_parquet"), "bytes")


CSV Size: 7224 bytes
JSON Size: 6089 bytes
Parquet Size: 17421 bytes


In [0]:
#5. Add a column total_revenue = price * quantity for each record.
df = df.withColumn("total_revenue", col("price") * col("quantity"))
df.show()

+----------+------------+-----------+-----+--------+-------------+
|product_id|product_name|   category|price|quantity|total_revenue|
+----------+------------+-----------+-----+--------+-------------+
|       101|      Laptop|Electronics|55000|      10|       550000|
|       102|  Smartphone|Electronics|30000|      25|       750000|
|       103|       Chair|  Furniture| 2500|      50|       125000|
|       104|        Book| Stationery|  400|     200|        80000|
|       105|  Headphones|Electronics| 1500|     100|       150000|
|       106|       Table|  Furniture| 3200|      40|       128000|
|       107|         Pen| Stationery|   20|     500|        10000|
|       108|     Monitor|Electronics|12000|      15|       180000|
|       109|    Notebook| Stationery|   60|     300|        18000|
|       110|        Sofa|  Furniture|45000|       5|       225000|
+----------+------------+-----------+-----+--------+-------------+



In [0]:
#6. Find the top 3 products with the highest total revenue.
df.orderBy(col("total_revenue").desc()).show(3)


+----------+------------+-----------+-----+--------+-------------+
|product_id|product_name|   category|price|quantity|total_revenue|
+----------+------------+-----------+-----+--------+-------------+
|       102|  Smartphone|Electronics|30000|      25|       750000|
|       101|      Laptop|Electronics|55000|      10|       550000|
|       110|        Sofa|  Furniture|45000|       5|       225000|
+----------+------------+-----------+-----+--------+-------------+
only showing top 3 rows


In [0]:
#7. Filter and display only Furniture products with price > 3000.
df.filter((col("category") == "Furniture") & (col("price") > 3000)).show()


+----------+------------+---------+-----+--------+-------------+
|product_id|product_name| category|price|quantity|total_revenue|
+----------+------------+---------+-----+--------+-------------+
|       106|       Table|Furniture| 3200|      40|       128000|
|       110|        Sofa|Furniture|45000|       5|       225000|
+----------+------------+---------+-----+--------+-------------+



Create a new column price_band with values:
'High' if price > 10000
'Medium' if 3000 < price <= 10000
'Low' if price ≤ 3000

In [0]:

df = df.withColumn(
    "price_band",
    when(col("price") > 10000, "High")
    .when((col("price") > 3000) & (col("price") <= 10000), "Medium")
    .otherwise("Low")
)
df.select("product_name", "price", "price_band").show()


+------------+-----+----------+
|product_name|price|price_band|
+------------+-----+----------+
|      Laptop|55000|      High|
|  Smartphone|30000|      High|
|       Chair| 2500|       Low|
|        Book|  400|       Low|
|  Headphones| 1500|       Low|
|       Table| 3200|    Medium|
|         Pen|   20|       Low|
|     Monitor|12000|      High|
|    Notebook|   60|       Low|
|        Sofa|45000|      High|
+------------+-----+----------+



In [0]:
#9. Group by category and calculate total quantity sold.
df.groupBy("category").sum("quantity").withColumnRenamed("sum(quantity)", "total_quantity").show()


+-----------+--------------+
|   category|total_quantity|
+-----------+--------------+
|Electronics|           150|
|  Furniture|            95|
| Stationery|          1000|
+-----------+--------------+



In [0]:
#10. Calculate average price of products for each category.
df.groupBy("category").avg("price").withColumnRenamed("avg(price)", "average_price").show()


+-----------+-------------+
|   category|average_price|
+-----------+-------------+
|Electronics|      24625.0|
|  Furniture|      16900.0|
| Stationery|        160.0|
+-----------+-------------+



In [0]:
#11. Count how many products fall in each price_band .
df.groupBy("price_band").count().show()


+----------+-----+
|price_band|count|
+----------+-----+
|      High|    4|
|       Low|    5|
|    Medium|    1|
+----------+-----+



In [0]:
#12. Write the filtered Electronics products (price > 5000) into a Parquet file.
df.filter((col("category") == "Electronics") & (col("price") > 5000)) \
  .write.mode("overwrite").parquet("dbfs:/tmp/electronics_filtered")


In [0]:
#13. Write the Stationery products into a JSON file.
df.filter(col("category") == "Stationery") \
  .write.mode("overwrite").json("dbfs:/tmp/stationery_products")


In [0]:
#14. Load Parquet back and run a query to find which category has highest total revenue.
df_parquet = spark.read.parquet("dbfs:/tmp/products_parquet")
df_parquet = df_parquet.withColumn("total_revenue", col("price") * col("quantity"))

df_parquet.groupBy("category") \
    .sum("total_revenue") \
    .orderBy(col("sum(total_revenue)").desc()) \
    .show(1)


+-----------+------------------+
|   category|sum(total_revenue)|
+-----------+------------------+
|Electronics|           1630000|
+-----------+------------------+
only showing top 1 row
